# Calculate the Tf-Idf Scores of Ngrams within a Document or Project 

Tf-idf, or term frequency - inverse document frequency, is a common way of measuring the importance of tokens both within a given document and across your project as a whole. You calculate a token's tf-idf score by multipling its relative frequency within a given document by the inverse of the number of documents that token appears in throughout the corpus. See <a href="https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089" target="_blank">TF-IDF from scratch in python on real world dataset</a> for a more in-depth explanation of the math.

Generally speaking, tokens with higher tf-idf scores (those closer to 1) are more important to a given document or corpus. At the document level, "distinctive" is a rough synonym for "important;" tf-idf provides a way to discover the tokens that are most distinctive within each document in your project. At the corpus or project level, a higher average tf-idf score means that a token is more frequently a distinctive word for documents within your corpus, i.e., it is potentially an important token for understanding your corpus overall.

### Technical Note

This notebook uses the NLTK package to build a custom tokenizer to tokenize project uni-, bi-, and trigrams. This tokenizer differs from the one used in the WE1S preprocessing pipeline. See the module's <a href="README.md" target="_blank">README.md</a> file for more information.

### INFO

__author__    = 'Lindsay Thomas'  
__copyright__ = 'copyright 2020, The WE1S Project'  
__license__   = 'MIT'  
__version__   = '2.0'  
__email__     = 'lindsaythomas@miami.edu'

## Settings

In [ ]:
# Python imports
import os
import csv
from pathlib import Path
from IPython.display import display, HTML

# Import scripts
%run scripts/count_tokens.py

# Define paths
current_dir     = %pwd
current_pathobj = Path(current_dir)
project_dir     = str(current_pathobj.parent.parent)
project_name    = os.path.basename(project_dir)
current_reldir  = current_dir.split("/write/")[1]
data_dir        = project_dir + '/project_data'
json_dir        = project_dir + '/project_data/json'
stopword_file   = '/home/jovyan/write/pub/templates/project_template/modules/topic_modeling/scripts/we1s_standard_stoplist.txt'

display(HTML('<p style="color: green;"><strong>Setup complete.</strong></p>'))

## 1. Configure Code

You must run all of the cells in this "Configure Code" section, even if you do not change the values.

### Set Content Field

In this cell, you will configure which field you would like to use as your input using the `content_field` variable. If your data includes full-text data in the `content` field, you can set `content_field = 'content'`. If you want to calculate frequency values using data in your `bag_of_words` field, set `content_field = 'bag_of_words'`. If you want to use the `features` field, set `content_field = 'features'`. Follow this format exactly (including the quotation marks).

**Please note that if you use EITHER `bag_of_words` OR `features` as your text input field, you will only be able to count unigrams. See the module's <a href="README.md" target="_blank">README.md</a> file for more information on these fields.**

In [ ]:
# You may select 'content', 'features', or 'bag_of_words'
content_field = 'content'

if content_field not in ['content', 'features', 'bag_of_words']:
    msg = "The <code>content_field</code> variable must be 'content', 'features', or 'bag_of_words'."
    display(HTML('<p style="color: red;">' + msg + '</p>'))
else:
    msg = 'You have set the <code>content_field</code> variable to <code>' + content_field + '</code>.'
    display(HTML('<p style="color: green;">' + msg + '</p>'))

### Set Tokenization Length
Configure the `set_length` variable below according to the length of ngram you are analyzing. The default is unigrams; to count bigrams or trigrams, comment out the unigram line, and uncomment the bi- or trigram line. 

**Note:** Because this code does not strip hyphens, hyphenated words like "first-generation" are considered unigrams.

In [ ]:
# Choose to analyze unigrams, bigrams, or trigrams
set_length = 'unigram'
# set_length = 'bigram'
# set_length = 'trigram'


if set_length not in ['unigram', 'bigram', 'trigram']:
    display(HTML("<p style=\"color: red;\">The <code>set_length</code> variable must be 'unigram', 'bigram', or 'trigram'.</p>"))
elif (set_length == 'bigram' or set_length == 'trigram' and content_field == 'bag_of_words') or (set_length == 'bigram' or set_length == 'trigram' and content_field == 'features'):
    msg = 'You cannot search for ' + set_length + 's with selected content field.'
    display(HTML('<p style="color: red;">' + msg + '</p>'))
else:
    msg = 'You have set the <code>set_length</code> variable to <code>' + set_length + '</code>.'
    display(HTML('<p style="color: green;">' + msg + '</p>'))

### Configure Punctuation Setting

This cell strips common punctuation from project documents. It will **NOT** strip hyphens, single or double, in order to account for hyphenated words and phrases such as "first-generation". Because this punctuation list is bespoke and not standardized (standardized options strip hyphens), some punctuation marks or other non-Unicode characters may make it through. You do not need to change anything about the below cell (unless you are interested in the frequency of punctuation marks, or @ signs, etc.), but you do need to run it. If you do not want to remove punctuation from your documents, you should set the `punctuations` variable to an empty string by uncommenting the line that says `punctuations = ''` in the cell below.

In [ ]:
# Define punctuation to strip
punctuations = "_______________________\'m\'n\'ve!()[]{};:\'\"\,<>./?@#$%^&*_~''``''"

# To strip no punctuation, uncomment the line below
# punctuations = ''

if punctuations == '':
    display(HTML('<p style="color: red;">You have elected not to strip any punctuation.</p>'))
else:
    msg = 'You have set the <code>punctuations</code> variable to <code>' + punctuations + '</code>.'
    display(HTML('<p style="color: green;">' + msg + '</p>'))   

### Configure Stop Word Setting

The default setting is to delete stop words from your data using the WE1S standard stoplist. You can view this list in you project's `modules/topic_modeling/scripts` folder. You can edit this file for your project or create a custom stoplist. If you use a custom list, make sure that it is a plain text file with one word per line. Upload the file to your project and configure the `stopword_file` variable in the **Settings** cell to indicate the path to your custom stop word file.

If your data has already had the stop words you want removed or if you do not want to remove stop words, change the value of `set_stopwords` to `False`.

It is generally recommended to delete stop words from a document before obtaining bi- and/or trigram frequencies. This will result in "inexact" bi- and trigrams, however, as any stop words will be deleted *before* tokenization into bi- or trigrams. If you are interested in specific bi- or trigrams that contain stop words, such as "first" in "first generation" (without a hyphen), you may want to create a custom stop word list.

In [ ]:
# Delete stopwords from content fields before obtaining word frequencies.
# If set to True, stop words will be deleted. If set to false, stop words will not be deleted.
set_stopwords = True

if set_stopwords == True:
    display(HTML('<p style="color: green;">You have elected to strip stopwords.</p>'))
else:
    display(HTML('<p style="color: red;">You have elected not to strip stopwords.</p>'))

## 2. Calculate Tf-Idf Scores


In [ ]:
# Obtain tf-idf scores
df_tfidf, vectors, feature_names, file_list, bad_jsons = tfidf_dir(json_dir, content_field, set_stopwords, 
                                                                   punctuations, set_length, stopword_file)

if len(bad_jsons) > 0:
        msg = 'Calculations complete. Warning! ' + str(len(bad_jsons)) + ' documents failed to load and will not be included in the calculation. '
        msg += 'If this number is large, this may significantly affect your results.'
        display(HTML('<p style="color: red;">' + msg + '</p>'))

if df_tfidf is not None:
    msg = 'Calculations complete. Total tokens in your project: <code>' + str(len(df_tfidf.columns)) + '</code>'
    display(HTML('<p style="color:green;">' + msg + '</p>'))
if df_tfidf.empty:
    display(HTML('<p style="color: red;">No results found.</p>'))


## 3. Explore Results
The cells below rely on the calculations you performed in the previous cell. You must run the previous cell before you can run any cells below, but all subsections below are optional and can be run in any order.

### Find the Average Tf-Idf Scores of All Tokens in the Project

To understand what words are important to your project overall, we can find the average tf-idf scores of all tokens in the project. By listing the tokens with the top n tf-idf scores across the project as a whole, we can understand what the most important words are across the whole project. Run the cell below to select the number of top results you would like to see. The results will be sorted from highest to lowest, and the default is 50. If you would like to change this, please change the value of the `top_n` variable below.

In [ ]:
# Select number of top results to display
top_n = 50

Get average tf-idf scores for top_n tokens.

In [ ]:
# Get top average tf-idf scores
df = top_mean_feats(vectors, feature_names, top_n, grp_ids=None)

In the next cell, you can view a dataframe listing the average tf-dif scores for the the top_n tokens in your project. 

The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
# Display dataframe
qgrid_widget = qgrid.show_grid(df, grid_options=grid_options, show_toolbar=False)

qgrid_widget

The cells below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the second cell below, uncomment the code in the cell below it, and run that cell. 

First, provide a name for the csv file the below code will create by providing a filename in the `csv_file` variable.

In [ ]:
# The filename of the csv file the below code will create
csv_file = ''

In [ ]:
# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file_counts, index_label = 'Index')

In [ ]:
# # Save the above dataframe to a csv file
# df_frequencies.to_csv(csv_file_counts, index_label = 'Index')

### Find the Tf-Idf Value of a Specific Token across All Project Documents

You could at this point print out a dataframe of all of the tf-idf scores across the whole project. But that would be a huge dataframe, and many of its cells would be empty because not every token across the project appears in every document. Instead, you can zero in on specific tokens you are interested in. You can use the below cells to find tf-idf scores for a specific token across all documents in your project. Check the `set_length` variable to make sure you have tokenized on the proper token length. I.e., if you are searching for a specific word, you should have tokenized on unigrams above when you ran the `tfidf_dir` function in the previous cells. If you ran the function using the unigram setting, but would now like to search for a bigram or trigram, you must reconfigure the `set_length` variable in **Section 1** of this notebook and rerun the above cell with the new configuration.

In [ ]:
# Check set_length variable
msg = 'You have set the <code>set_length</code> variable to <code>' + set_length + '</code>.'
display(HTML('<p style="color: green;">'+ msg + '</p>'))

Enter the token you want to search for. If you are searching for a unigram, just enter it below. The tokenization method used in this notebook is case-insensitive (i.e., it is not possible to differentiate between "book" and "Book"). If you are searching for a specific bigram or trigram, enter it with spaces separating each part. Remember that if you deleted stopwords above, your bi- or trigram should not contain stopwords. Follow the below format exactly:

`token = 'unigram'`

OR

`token = 'bigram example'`

OR

`token = 'trigram example here'`


In [ ]:
# Configure token
token = ''

Get tf-idf scores for that token. If your token as it is entered above does not appear in the project, this cell will print a statement notifying you.

In [ ]:
# Get tf-idf scores
df_tfidf_token = tfidf_token(set_length, token, df_tfidf)

In the next cell, you can view a dataframe listing the filenames of the documents in which your selected token appears and its tf-idf score for each document in which it appears. 

The below cell uses a <a href="https://github.com/quantopian/qgrid" target="_blank">QGrid</a> widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
# Display dataframe
qgrid_widget = qgrid.show_grid(df_tfidf_token, grid_options=grid_options, show_toolbar=False)

qgrid_widget

The cells below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the second cell below, uncomment the code in the cell below it, and run that cell. 

First, provide a name for the csv file the below code will create by providing a filename in the `csv_file` variable.

In [ ]:
# The filename of the csv file the below code will create
csv_file = ''

In [ ]:
# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file_counts, index_label = 'Index')

In [ ]:
# # Save the above dataframe to a csv file
# df_frequencies.to_csv(csv_file_counts, index_label = 'Index')

### Find the Tokens with the Top Tf-Idf Scores in a Specific Document

We can also find the top_n tf-idf scores for any given document in your project. To do this, you must enter **the full file path** of the specific document you want to analyze: e.g. `fpath='foo/bar/project_name/project_data/json/document.json'`.

In [ ]:
# Define file path for document you want to explore in more detail
fpath = ''

if (fpath == '' or fpath == None):
    display(HTML('<p style="color: red;">You have not provided a specific file to analyze.</p>'))
else:
    display(HTML('<p style="color: green;">You have selected the following file: <code>' + fpath + '</code>.</p>'))    

Run the cell below to select the number of tf-idf scores for your selected document that you would like to see. The results will be sorted from highest to lowest, and the default is 50. If you would like to change this, please change the value of the `top_n` variable below.

In [ ]:
# Set number of tokens to see
top_n = 50

In [ ]:
# Get row index
row_id = file_list.index(fpath)

# Get top tf-idf scores for the specific document
df_single = top_feats_in_doc(vectors, feature_names, row_id, top_n)

In the next cell, you can view a dataframe listing the top tf-idf scores for your selected document. 

The below cell uses a <a href="https://github.com/quantopian/qgrid" target="_blank">QGrid</a> widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
# Display dataframe
qgrid_widget = qgrid.show_grid(df_single, grid_options=grid_options, show_toolbar=False)

qgrid_widget

The cells below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the second cell below, uncomment the code in the cell below it, and run that cell. 

First, provide a name for the csv file the below code will create by providing a filename in the `csv_file` variable.

In [ ]:
# The filename of the csv file the below code will create
csv_file = ''

In [ ]:
# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file_counts, index_label = 'Index')

In [ ]:
# # Save the above dataframe to a csv file
# df_frequencies.to_csv(csv_file_counts, index_label = 'Index')